In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
import csv
from pathlib import Path

import pandas as pd

In [ ]:
SUBMISSION = "RSNA_20220214"

PACKAGES_PATH = Path(f"/Users/andrewprokhorenkov/CTDS/projects/midrc/midrc_indexing_scripts/packages/packages_{SUBMISSION}")
PACKAGES_PATH.mkdir(parents=True, exist_ok=True)

RSNA_PATH = Path("/Users/andrewprokhorenkov/CTDS/projects/midrc/s3-data/raw/rsna")
SUBMISSION_PATH = RSNA_PATH / SUBMISSION

manifest_file = list(SUBMISSION_PATH.glob("imaging_data_manifest_*.tsv"))[0]
instance_files = list(SUBMISSION_PATH.glob("*_instance_*.tsv"))
# instance_files = list(SUBMISSION_PATH.glob("midrc_*_instance_*.tsv"))
# instance_files = list(SUBMISSION_PATH.glob("midrc_*_image_*.tsv"))
studies_file = list(SUBMISSION_PATH.glob("imaging_study_*.tsv"))[0]
# studies_file = list(SUBMISSION_PATH.glob("midrc_imaging_study_*.tsv"))[0]

manifest = pd.read_csv(manifest_file, sep="\t")

instances = list(map(lambda v: pd.read_csv(v, sep="\t"), instance_files))

rename_columns = {'cr_series.submitter_id': 'series_id',
'dx_series.submitter_id': 'series_id',
'ct_series.submitter_id': 'series_id',
'mr_series.submitter_id': 'series_id'}

instances = list(map(lambda v: v.rename(columns=rename_columns), instances))

all_instances = pd.concat(instances)

studies = pd.read_csv(studies_file, sep="\t")
studies["case_id"] = studies["cases.submitter_id"]
# studies["case_id"] = studies["cases.submitter_id"].apply(lambda v: v.split("_")[1])
# studies["case_id"] = studies["cases"].apply(lambda v: v.split("_")[1])

merged = manifest.merge(all_instances, how="inner", on="md5sum", suffixes=["", "_y"])

merged["case_id"] = merged["case_ids"]
# merged["case_id"] = merged["case_ids"].apply(lambda v: v.split("_")[1])
merged["instance_id"] = merged["submitter_id"]

# merged["case_id"] = merged["instance.submitter_id"].apply(lambda v: v.split("_")[2])
# merged["series_id"] = merged["instance.submitter_id"].apply(lambda v: v.split("_")[3])
# merged["instance_id"] = merged["submitter_id"].apply(lambda v: v.split("_")[3])

# merged["case_id"] = merged["submitter_id"].apply(lambda v: v.split("_")[0])
# merged["series_id"] = merged["series_uid"]
# merged["instance_id"] = merged["submitter_id"].apply(lambda v: v.split("_")[1])

merged["file_name"] = merged["instance_id"].apply(lambda v: f"{v}.dcm")

In [ ]:
merged.columns

In [ ]:
studies.columns

In [ ]:
merged.iloc[0]

In [ ]:
merged_final = merged.reset_index().merge(studies.reset_index(), on="case_id", suffixes=["", "_y"])

merged_final = merged_final[["file_name", "file_size", "md5sum", "storage_urls", "case_id", "study_uid", "series_id"]]

merged_final

In [ ]:
for i, row in merged.iterrows():
    case_id = row["case_id"]
    study_id = row["study_uid"]
    series_id = row["series_id"]

    folder = PACKAGES_PATH / case_id / study_id

    folder.mkdir(parents=True, exist_ok=True)
    
    series_file = folder / f"{series_id}.tsv"
    series_file_exist = series_file.exists()

    with open(series_file, mode="a") as f:
        fieldnames = ['file_name', 'file_size', 'md5sum', 'case_id', 'study_id', 'series_id', 'instance_id', 'storage_urls']
        writer = csv.DictWriter(f, delimiter="\t", fieldnames=fieldnames)

        if not series_file_exist:
            writer.writeheader()
        writer.writerow(row.to_dict())